In [1]:
import numpy as np
import open3d as o3d
import utilities as uti
from plyfile import PlyData, PlyElement

import sys
sys.path.insert(1, './Colors')
import realman_gaussian_color as rgc

curvarute_type = 'gauss'
cam = np.array([0,0,0])
height = 200
width = 100

data = np.zeros((height, width,2))
data[:,:,0] = None

In [2]:
from PIL import Image
def projectImage(data):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    frame = frame + 255
    for i in range(height):
        for j in range(width):
            if not np.isnan(data[i][j]):
                color = rgc.getColor(data[i][j])
                frame[i][j] = np.array(color[0:3])
    img = Image.fromarray(frame, 'RGB')
    #img.save('my.png')
    img.show()

def computeAverageCurvature(data):
    curvatures = data[:,:,0]
    count = data[:,:,1]
    for i in range(height):
        for j in range(width):
            if not np.isnan(curvatures[i][j]):
                curvatures[i][j] = curvatures[i][j]/count[i][j]
    return curvatures

In [3]:
# Main loop
for i in range(3,4):
    plydata = PlyData.read('./Mesh/results/jean normal/nonback_'+curvarute_type+'_pose_'+str(i)+'.ply')
    points = np.zeros((plydata['vertex'].count,2))
    values = plydata['vertex']['quality']
    points[:,0] = plydata['vertex']['x']
    points[:,1] = plydata['vertex']['y']
    center = [np.average(points[:,0]),np.average(points[:,1])]
    points[:,0] = np.around((points[:,0] - center[0])*width + width/2)
    points[:,1] = np.around((points[:,1] - center[1])*width + width)

    for i in range(len(values)):
        if np.isnan(data[height-int(points[i][1])][width-int(points[i][0])][0]):
            data[height-int(points[i][1])][width-int(points[i][0])][0] = values[i]
        else:
            data[height-int(points[i][1])][width-int(points[i][0])][0] += values[i]
        data[height-int(points[i][1])][width-int(points[i][0])][1] += 1

In [4]:
result = computeAverageCurvature(data)
projectImage(result)

# importing pandas as pd 
import pandas as pd 

# Creating the dataframe 
df = pd.DataFrame(data=data)
# for i in df:
# 	df[i] = df[i].interpolate(method ='cubic')
# inter_data = df.interpolate(method ='linear', limit_direction ='backward').to_numpy()